In [1]:
import os
import PyPDF2
import pdfplumber
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
import joblib

In [3]:
def extract_text_and_features_from_pdf(file_path):
    text = ""
    num_pages = 0
    has_table = False
    keyword_count = {"factura": 0, "total": 0, "contrato": 0, "cláusula": 0, "firma": 0, "cufe": 0}
    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)
        for page in pdf.pages:
            text += page.extract_text()
            if page.extract_tables():
                has_table = True
    for keyword in keyword_count.keys():
        keyword_count[keyword] = text.lower().count(keyword)
    features = [num_pages, int(has_table)] + list(keyword_count.values())
    return features

In [10]:
labels = []
features_list = []
for category in ['factura', 'contrato']:
    folder_path = os.path.join('documentos_train', category)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        features = extract_text_and_features_from_pdf(file_path)
        labels.append(category)
        features_list.append(features)

In [11]:
df_feature = pd.DataFrame(features_list, columns=['num_pages', 'has_table', 'factura', 'total', 'contrato', 'cláusula', 'firma','cufe'])
df_label = pd.DataFrame(labels, columns=['category'])
df = pd.concat([df_feature, df_label], axis=1)
df

num_pages  has_table  factura  total  contrato  cláusula  firma  cufe  \
0           1          1        3      5         0         0      1     1   
1           1          1        5      3         0         0      2     1   
2           1          1        2      8         0         0      0     1   
3           1          1        5      3         0         0      2     1   
4           2          1        5      4         0         0      2     1   
5           1          1        4      4         0         0      0     1   
6           2          1        6      5         0         0      0     0   
7           2          1        5      2         0         0      2     1   
8           1          1        3      6         0         0      1     1   
9           1          1        3      6         0         0      1     1   
10          1          1        8      8         0         0      1     1   
11          1          1        3      5         0         0      0     1   
12          1          1        2     11         0         0      0     1   
13          2          1        6      5         0         0      0     0   
14          1          1        2      4         0         0      1     1   
15          1          1        3      5         0         0      0     1   
16          1          1        3      5         0         0      0     1   
17          1          1        3      5         0         0      0     1   
18          1          1        5      6         0         0      1     1   
19          1          0        3      1         0         0      1     0   
20          1          1        7      2         0         0      0     1   
21          1          1        7      2         0         0      0     1   
22          1          1        4      4         0         0      0     1   
23          1          1        3      5         0         0      0     1   
24          1          1        4      3         0         0      0     1   
25          1          1        7      2         0         0      0     1   
26          1          1        6      3         0         0      1     1   
27          1          1        7      2         0         0      0     1   
28          3          0        0      3        10         1      4     0   
29          4          0        0      1         9         1      3     0   
30          2          0        0      0        13         0      0     0   
31         25          1        8     18       220        68     11     0   
32          3          0        0      0         8         2      1     0   
33          3          0        0      1         8         1      2     0   
34          2          0        0      2        16         1      4     0   
35          2          0        0      0         0         2      1     0   
36          7          1        1      8        58        10      4     0   
37          2          0        0      0         9         2      1     0   
38          3          0        0      4        12         3      5     0   
39          3          0        0      1        10         1      5     0   
40          4          0        0      2        11         1      2     0   

    category  
0    factura  
1    factura  
2    factura  
3    factura  
4    factura  
5    factura  
6    factura  
7    factura  
8    factura  
9    factura  
10   factura  
11   factura  
12   factura  
13   factura  
14   factura  
15   factura  
16   factura  
17   factura  
18   factura  
19   factura  
20   factura  
21   factura  
22   factura  
23   factura  
24   factura  
25   factura  
26   factura  
27   factura  
28  contrato  
29  contrato  
30  contrato  
31  contrato  
32  contrato  
33  contrato  
34  contrato  
35  contrato  
36  contrato  
37  contrato  
38  contrato  
39  contrato  
40  contrato

In [14]:
X = df.drop('category', axis=1)
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
joblib.dump(classifier, 'modelo_entrenado.pkl')

['modelo_entrenado.pkl']

In [19]:
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 1.00


In [21]:
report = classification_report(y_test, y_pred)
print("\nReporte de clasificación:")
print(report)


Reporte de clasificación:
              precision    recall  f1-score   support

    contrato       1.00      1.00      1.00         3
     factura       1.00      1.00      1.00         6

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



In [22]:
import shutil
import warnings
warnings.filterwarnings('ignore')
classifier = joblib.load('modelo_entrenado.pkl')

In [30]:
def preprocess_and_predict(file_path, model):
    features = extract_text_and_features_from_pdf(file_path)
    prediction = model.predict(np.array([features]))
    return prediction

In [31]:
def move_file_based_on_prediction(file_path, prediction):
    os.makedirs('Documentos/carpeta_facturas', exist_ok=True)
    os.makedirs('Documentos/carpeta_contratos',exist_ok=True)
    if prediction[0] == 'factura':
        destination_path = 'Documentos/carpeta_facturas'
        shutil.copy(file_path, destination_path)
    else:
        destination_path = 'Documentos/carpeta_contratos'
        shutil.copy(file_path, destination_path)
    print(f"El archivo {file_path} ira {destination_path}")

In [33]:
directorio = './documentos_prueba/'
for archivo in os.listdir(directorio):
    prediccion = preprocess_and_predict(directorio+archivo, classifier)
    print(f"El archivo {archivo} es de tipo {prediccion[0]}")
    move_file_based_on_prediction(directorio+archivo, prediccion)

El archivo aa.pdf es de tipo contrato
El archivo ./documentos_prueba/aa.pdf ira Documentos/carpeta_contratos
El archivo ad089090060802724797520241023183345600.pdf es de tipo factura
El archivo ./documentos_prueba/ad089090060802724797520241023183345600.pdf ira Documentos/carpeta_facturas
El archivo fv0800242106021249937100040573.pdf es de tipo factura
El archivo ./documentos_prueba/fv0800242106021249937100040573.pdf ira Documentos/carpeta_facturas
El archivo bb.pdf es de tipo contrato
El archivo ./documentos_prueba/bb.pdf ira Documentos/carpeta_contratos
El archivo fv0900944634004242ca620cd.pdf es de tipo factura
El archivo ./documentos_prueba/fv0900944634004242ca620cd.pdf ira Documentos/carpeta_facturas
El archivo cc.pdf es de tipo contrato
El archivo ./documentos_prueba/cc.pdf ira Documentos/carpeta_contratos
El archivo 900413476_FE_FE2107088.pdf es de tipo factura
El archivo ./documentos_prueba/900413476_FE_FE2107088.pdf ira Documentos/carpeta_facturas
